# Desafío 2. Prediciendo precios de propiedades

**Grupo 2:**

Faro, Gonzalo

Cabrol, Angelica

Vinyolas, Mariana

Hutler, Ianina



# [Índice](#Índice)

- [Objetivos](#Objetivos)
- [Indicaciones](#Indicaciones)
- [Importación de librerías](#Importación-de-librerías)
- [Seteos Generales](#Seteos-Generales)
- [Feature Selection](#Feature-Selection)
- [Análisis de propiedades en Argentina](#Análisis-de-Propiedades-en-Argentina)
- [Análisis de propiedades en CABA](#Análisis-de-Propiedades-en-CABA)
- [Análisis de propiedades en GBA](#Análisis-de-Propiedades-en-GBA)
- [Análisis de propiedades en Provincia De Buenos Aires](#Análisis-de-Propiedades-en-Provincia-de-Buenos-Aires)
- [Análisis de propiedades en Patagonia](#Análisis-de-Propiedades-en-Patagonia)
- [Análisis de propiedades en Región Pampeana](#Análisis-de-Propiedades-en-Región-Pampeana)
- [Análisis de propiedades en Noroeste](#Análisis-de-Propiedades-en-Noroeste)
- [Análisis de propiedades en Nordeste](#Análisis-de-Propiedades-en-Nordeste)
- [Análisis de propiedades en Cuyo](#Análisis-de-Propiedades-en-Cuyo)
- [Análisis de Sample en CABA](#Análisis-de-Sample-en-CABA)

# [Objetivos](#Objetivos)
[Volver](#Índice)

Objetivos:

● Estimar un modelo de regresión lineal que realice predicciones para el precio por metro cuadrado. Deberá prestar cierta atención a la estructura espacial de los precios.

● Aplicar regularización a modelos lineales (pueden hacerlo para obtener un puntaje adicional). La idea es la siguiente: estimar una regresión Ridge y una LASSO sobre el dataset. Para ello, deberán usar cross-validation para tunear el parámetro de regularización que maximiza R2 en tu test set. ¿Cómo son las performances entre los modelos regularizados y no regularizado? ¿Cuál funciona mejor? ¿Qué hace una regresión Ridge? ¿Y una LASSO?¿Qué diferencias hay con la regresión lineal sin regularizar?

● Seleccionar mediante muestreo aleatorio simple una submuestra de 100 propiedades. Este será su portafolio de departamentos. En base al mejor modelo que haya encontrado determine cuáles de los departamentos, tanto en su portafolio como fuera de él, se encuentran sobrevaluados o subvaluados y en qué magnitud.

● Teniendo en cuenta que podría comprar y vender propiedades al precio de mercado, omitamos costos de transacción, con un capital inicial igual al valor de mercado de las propiedades en su portafolio. ¿Cuál es el mejor portafolio de propiedades que podría comprar?

# [Indicaciones](#Indicaciones)
[Volver](#Índice)

Este trabajo se organizará de la siguiente manera:

1- Análisis de propiedades en Argentina.

2- Análisis de propiedades en CABA.

2.1 Análisis de propiedades en CABA por tipo de propiedad.

3- Análisis de propiedades en el Gran Buenos Aires (gba).

4- Análisis de propiedades en Bs As provincia.

5- Análisis de propiedades en Patagonia.

6- Análisis de propiedades en la región pampeana.

7- Análisis de propiedades en la región noroeste.

8- Análisis de propiedades en la región nordeste. 

9- Análisis de propiedades en la región de Cuyo.

10- Conclusiones

# [Importación de librerías](#Importación-de-librerías)
[Volver](#Índice)

**Importamos los diferentes paquetes a utilizar.**

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.simplefilter('ignore', category=FutureWarning)

# [Seteos Generales](#Seteos-Generales)
[Volver](#Índice)

**Seteos generales**

In [ ]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 200)

# [Feature Selection](#Feature-Selection)
[Volver](#Índice)

**Feature selection:**

Para predecir el precio por metro cuadrado de las propiedades de Argentina seleccionamos como relevantes las siguientes features: tamaño de la propiedad (superficie total en m2), el tipo de propiedad (PH, departamento, negocio o casa), cantidad de ambientes (rooms), ubicación (barrio, provincia o localidad), amenities (pileta, parrila, gimnasio, sauna, laundry, playroom, sum, sala de reuniones, restaurant y quincho), si tiene o no superficie descubierta (terraza, balcón o patio), si tiene o no expensas, si tiene garage o cochera, si tiene o no vigilancia, si tiene o no dependencia, si es a estrenar, en refacción o normal (ninguna de las dos anteriores) y si se encuentra en un barrio cerrado o country. 

# [Análisis de Propiedades en Argentina](#Análisis-de-Propiedades-en-Argentina)
[Volver](#Índice)

## Análisis de propiedades en Argentina. ##


**Cargamos el DataFrame de Argentina**

In [ ]:
df_arg = pd.read_csv('df_total.csv', index_col=0)
display(df_arg.shape) # Nos fijamos cuantas filas y columnas tiene
display(df_arg.sample(10)) # Selección de 10 ejemplares al azar para su observación
display(df_arg.isnull().sum()) # Visualización de las columnas con valores null

In [ ]:
quien_es = df_arg[df_arg['surface_total_in_m2']<=15]
quien_es['surface_total_in_m2'].sort_values()

Las columnas geonames_id, lat-lon, lat, lon, price_aprox_usd, localidad y barrio no van a ser utilizadas. 
La columna de provincia es la que vamos a utilizar para crear dummies y no tiene ningun valor null

Generamos Variables Dummies para las provincias:

In [ ]:
prov_dummies = pd.get_dummies(df_arg.provincia, prefix='Provincia')
df_arg = pd.concat([df_arg, prov_dummies], axis=1) 
display(df_arg.shape)
display(df_arg.sample(5))

Eliminamos las filas con valores nan para surface_total_in_m2, price_usd_per_m y rooms que son las que voy a conservar para el análisis.

In [ ]:
df_arg = df_arg[(df_arg.surface_total_in_m2.notnull())\
                  & (df_arg.price_usd_per_m2.notnull())\
                  & (df_arg.rooms.notnull())]
display(df_arg.shape)

**Analisis de la columna amenities**

Recuperamos por regex las siguientes amenities: pileta, parrilla, gimnasio, sauna, laundry, playroom, sum, sala de reuniones, restaurant y quincho. Creamos un contador y vemos con cuantos amenities cuenta la propiedad. Decidimos entonces eliminar las amenities por separado y quedarnos con el contador. A continuación también eliminamos el resto de las columnas que no va a participar en el análisis. Esto se repetirá en todas las regiones de Argentina.

In [ ]:
df_arg.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 'price_aprox_usd', 'pileta', 
         'parrilla', 'gimnasio', 'sauna', 'expenses', 'laundry', 'playroom', 'sum', 'sala_reuniones', 'restaurant',
         'quincho'], inplace=True, axis=1)
display(df_arg.shape)
display(df_arg.sample(5))
display(df_arg.describe())

#### Después de esta limpieza, para el analisis de regresión vamos a tener 41217 filas y 47 columnas

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_df = df_arg.drop('price_usd_per_m2', axis=1)
y_df = df_arg.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

Separamos los datos para train y test utilizando la función de Scikit-learn que se encarga de la separación entre test y entrenamiento. Seteamos el parámetro test_size en 0.3 que será la propoción de los datos que vamos a separar para hacer la evaluación.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Es importante estandarizar todos los regresores antes de ejecutar una regresión Ridge y Lasso. Así ya no están en unidades físicas sino en unidades de su propio desvío estándar.

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

***Sobre el set de entrenamiento (Train Set)***

Proceso de Tunning de los Hiperparámetros: Asignamos los alphas y definimos kfold como estrategia de cross validation.

In [ ]:
alphas = np.linspace(0.1, 10000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

Imprimimos el Score de entrenamiento de cada modelo.
El coeficiente de determinación, denominado R² representa cuánta varianza explica el modelo del total de varianza existente

In [ ]:
print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

Vemos que apesar de que el R2 es muy bajo, mejora levemente con el test y el RMSE baja. Vamos a probar el mismo procedimiento eliminando outliers.

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

Observación de outliers en Argentina

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))
import seaborn as sns
sns.boxplot(x=df_arg['price_usd_per_m2']) #observación de los outliers. 

In [ ]:
df_arg['price_usd_per_m2'].describe() 

Vemos que el cuantil 75 es razonable pero hay un maximo de $ 540.000 que podriamos filtrar para ver si el modelo mejora

In [ ]:
df_arg_out = pd.read_csv('df_total.csv', index_col=0) #importo nuevamente el archivo de argentina bajo otro nombre

In [ ]:
df=df_arg_out 
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 2: # cualquier punto fuera de 3 desviaciones estandard son considerados outlier.
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 2:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df=df[(df.outlier!='YES')]

In [ ]:
df.drop(['outlier'], inplace=True, axis=1)

Repetimos el procedimiento: Generamos Variables Dummies:

In [ ]:
prov_dummies = pd.get_dummies(df.provincia, prefix='Provincia')
df = pd.concat([df, prov_dummies], axis=1) 

In [ ]:
df = df[(df.surface_total_in_m2.notnull())\
        & (df.price_usd_per_m2.notnull())\
        & (df.rooms.notnull())]
display(df.shape)

In [ ]:
df['price_usd_per_m2'].describe() 

In [ ]:
quien_es = df[df['price_usd_per_m2']>=20000] 
quien_es

In [ ]:
df.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 'price_aprox_usd', 'pileta', 
         'parrilla', 'gimnasio', 'sauna', 'expenses', 'laundry', 'playroom', 'sum', 'sala_reuniones', 'restaurant',
         'quincho'],inplace=True, axis=1)
display(df.shape)
display(df.sample(5))

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_df = df.drop('price_usd_per_m2', axis=1)
y_df = df.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42) # Separamos los datos para train y test
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas
alphas = np.linspace(0.01, 1000, 300)
# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

Imprimimos el Score de entrenamiento de cada modelo y observamos nuevamente el R² y RMSE

In [ ]:
print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

**Conclusión:**
Luego de aplicar el filtro de los outliers tanto R2 como  RMSE se mantuvieron con el mismo valor aproximadamente

***--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## [Análisis de Propiedades en CABA](#Análisis-de-propiedades-en-CABA)
[Volver](#Índice)

### 2. Análisis de propiedades en CABA. ###

**Cargamos el DataFrame de CABA**

In [ ]:
df_caba = pd.read_csv('df_caba.csv', index_col=0)
display(df_caba.shape) #Nos fijamos con cuantas filas y columnas arrancamos
display(df_caba.sample(10)) #Seleccionamos un sample de 10 para que el muestreo sea aleatorio
display(df_caba.isnull().sum())#Verificamos si hay valores faltantes
display(df_caba['price_usd_per_m2'].describe()) #Observamos los estadisticos para la columna price_usd_per_m2 para ver posibles outliers

Debido a que en el DataFrame de Argentina el modelo de regresión mejoró gracias a eliminar algunos outliers decidimos implementar lo mismo.

In [ ]:
df=df_caba
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3:
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_caba=df[(df.outlier!='YES')]

In [ ]:
df_caba.drop(['outlier'], inplace=True, axis=1)
df_caba.head()

Las columnas geonames_id, lat-lon, lat, lon, price_aprox_usd, provincia y barrio no van a ser utilizadas. 
La columna de localidad es la que vamos a utilizar para crear dummies y no tiene ningun valor null.

Eliminamos las filas con valores nan para surface_total_in_m2, price_usd_per_m2,rooms y localidad que son las que voy a conservar.

In [ ]:
df_caba = df_caba[(df_caba.surface_total_in_m2.notnull())\
                  & (df_caba.price_usd_per_m2.notnull())\
                  & (df_caba.rooms.notnull())\
                  & (df_caba.localidad.notnull())]
display(df_caba.shape)
display(df_caba.isnull().sum()) 

In [ ]:
#### Como son muchas las dummies de amenities las eliminamos solo para ver el heatmap, dejando el contador amenities ####

df_heatmap = df_caba.drop(['amenities', 'geonames_id', 'lat-lon', 'lat','lon', 'provincia', 'barrio'], axis=1)

In [ ]:
df_heatmap.columns

In [ ]:
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (40, 25)
sns.set(font_scale=1.7)
sns.heatmap(df_heatmap.corr(), annot=True, fmt='.2f', cmap="BuPu");

Generamos variables dummies para los barrios de CABA. (dicha información viene contenida en la columna localidad).
Luego unimos estas dummies con el DataFrame de CABA

In [ ]:
barrios_dummies = pd.get_dummies(df_caba.localidad, prefix='barrio')
df_caba = pd.concat([df_caba, barrios_dummies], axis=1) 
df_caba.head()

In [ ]:
df_caba.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
display(df_caba.head())
display(df_caba.shape)

Por lo tanto, las columnas que vamos a utilizar son: surface_total_in_m2, price_usd_per_m2, rooms, terraza, balcón, patio, expensas, barrio_cerrado (que también incluye country),vigilancia, si tiene dependencia (habitación de servicio), estrenar, normal, refacción y cochera.

In [ ]:
# Cargamos un dataframe, en lugar de obtener el sample, para asegurarnos que haya al menos un registro
# por cada tipo de propiedad. Para generar una muestra en el momento, descomentar las siguientes 3 lineas
# y comentar las ultimas 2
# df_sample = df_caba.sample(100)
# df_caba = df_caba.drop( df_sample.index)
# df_sample.to_csv('caba_sample.csv')

df_sample = pd.read_csv('caba_sample.csv', index_col=0)
df_caba = df_caba.drop( df_sample.index)

In [ ]:
# Definimos X e y
X_df = df_caba.drop('price_usd_per_m2', axis=1)
y_df = df_caba.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas y definimos kf para utilizar kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 1000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set
y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

**Probamos los resultados sacando outliers y dividiendo por tipo de propiedad en CABA**

In [ ]:
df_caba = pd.read_csv('df_caba.csv', index_col=0) #Cargamos el df de CABA nuevamente 
df_caba = df_caba[(df_caba.surface_total_in_m2.notnull())\
                  & (df_caba.price_usd_per_m2.notnull())\
                  & (df_caba.rooms.notnull())\
                  & (df_caba.localidad.notnull())]
display(df_caba.shape)

In [ ]:
df_caba.price_usd_per_m2.describe()

In [ ]:
# analizamos las propiedades de Boedo
df_boedo = df_caba[(df_caba.localidad == 'Boedo')]
df_boedo.price_usd_per_m2.describe()

In [ ]:
#Se verifica que muchas de las propiedades tienen en precio en ARS, las pasamos a USD
df_boedo.price_usd_per_m2 = df_boedo.price_usd_per_m2.apply( lambda x: x / 17.6445 if x > 7000 else x )

In [ ]:
df_boedo.price_usd_per_m2.describe()

In [ ]:
df_caba = df_caba[(df_caba.localidad != 'Boedo')]
df_caba = pd.concat([df_caba, df_boedo], ignore_index=True)

In [ ]:
# Analizamos las propiedades de San Cristobal
df_sancri = df_caba[(df_caba.localidad == 'San Cristobal')]
df_sancri.price_usd_per_m2.describe()

In [ ]:
(df_sancri.price_usd_per_m2 > 7000).sum()

In [ ]:
# se verifica que muchas de las propiedades tienen en precio en ARS, las pasamos a USD
df_sancri.price_usd_per_m2 = df_sancri.price_usd_per_m2.apply( lambda x: x / 17.6445 if x > 7000 else x )

In [ ]:
df_sancri.price_usd_per_m2.describe()

In [ ]:
df_caba = df_caba[(df_caba.localidad != 'San Cristobal')]
df_caba = pd.concat([df_caba, df_sancri], ignore_index=True)
df_caba.price_usd_per_m2.describe()

In [ ]:
# Aplicamos una funcion para eliminar Outliers
df=df_caba
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3:
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"


In [ ]:
df_caba=df[(df.outlier!='YES')]

In [ ]:
df_caba.drop(['outlier'], inplace=True, axis=1)

**Generamos variables dummies para los barrios de CABA**

In [ ]:
barrios_dummies = pd.get_dummies(df_caba.localidad, prefix='barrio')
df_caba = pd.concat([df_caba, barrios_dummies], axis=1) 
display(df_caba.shape)

Eliminamos las columnas que no vamos a utilzar

In [ ]:
df_caba.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
display(df_caba.head())
display(df_caba.shape)

**Separamos el df según el tipo de propiedad**

In [ ]:
df_ph = df_caba[(df_caba.PH== 1)]
df_ph.drop(['apartment', 'store', 'house', 'PH'], inplace=True, axis=1)
df_apartment = df_caba[(df_caba.apartment== 1)]
df_apartment.drop(['PH', 'store', 'house', 'apartment'], inplace=True, axis=1)
df_store = df_caba[(df_caba.store== 1)]
df_store.drop(['PH', 'apartment', 'house','store'], inplace=True, axis=1)
df_house = df_caba[(df_caba.house== 1)]
df_house.drop(['apartment', 'store', 'PH', 'house'], inplace=True, axis=1)

display(df_ph.head())
display(df_ph.shape)
display(df_apartment.head())
display(df_apartment.shape)
display(df_store.head())
display(df_store.shape)
display(df_house.head())
display(df_house.shape)

**Modelos de Regresión**


Tipo de Propiedad: 'PH'

In [ ]:
# Definimos X e y
X_df = df_ph.drop('price_usd_per_m2', axis=1)
y_df = df_ph.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
scaler_ph = StandardScaler() #estandarizamos los datos
X_train = scaler_ph.fit_transform(X_train)
X_test = scaler_ph.transform(X_test)

In [ ]:
#Asignamos los alphas
alphas = np.linspace(0.1, 100, 300)
# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr_ph = lm.fit(X_train, y_train)
modelo_ridge_ph = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso_ph = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
pd.Series(y_test).describe()

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", fontsize='small')
plt.ylabel("Valores reales precio en USD por m2", fontsize='small')
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
#Distribución de la muestra
pd.Series(y_test).hist(bins=100, figsize=(20, 5))

Imprimimos los coeficientes para cada modelo: 'PH' Intercept y coef de cada feature

In [ ]:
####  Regresion Lineal  ####
print (lm.intercept_)
lm_coefs = list(zip(X_df.columns, lm.coef_))
lm_coefs =  sorted(lm_coefs, key=lambda x:x[1], reverse=True)


In [ ]:
# armamos un df con los coeficientes de LM
lm_coefs = pd.DataFrame(lm_coefs, columns=('Features', 'LM_Coef'))
lm_coefs = lm_coefs.set_index('Features')
lm_coefs

In [ ]:
####  Ridge con CV  ####
print (lm_ridge_cv.intercept_)
ridge_coefs = list(zip(X_df.columns, lm_ridge_cv.coef_))
ridge_coefs = sorted(ridge_coefs, key=lambda x:x[1], reverse=True)

In [ ]:
# armamos un df con los coeficientes de Ridge
ridge_coefs = pd.DataFrame(ridge_coefs, columns=('Features','Ridge_Coef'))
ridge_coefs = ridge_coefs.set_index('Features')
ridge_coefs


In [ ]:
####  LASSO con CV  ####
print (lm_lasso_cv.intercept_)
lasso_coefs = list(zip(X_df.columns, lm_lasso_cv.coef_))
lasso_coefs = sorted(lasso_coefs, key=lambda x:x[1], reverse=True)

In [ ]:
# armamos un df con los coeficientes de Lasso
lasso_coefs = pd.DataFrame(lasso_coefs, columns=('Features','Lasso_Coef'))
lasso_coefs = lasso_coefs.set_index('Features')
lasso_coefs


In [ ]:
# ploteamos los coeficientes de LM para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lm_coefs['LM_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Ridge para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
ridge_coefs['Ridge_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Lasso para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lasso_coefs['Lasso_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# Imprimimos un heatmap para evaluar las correlaciones de cada feature y los coef calculados con el target
sns.set(rc={'figure.figsize':(35,4)})
sns.heatmap(pd.concat([X_df, pd.DataFrame(y_df)], axis=1).corr().iloc[[-1]], annot=True);

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


**Tipo de Propiedad: 'APARTMENT'**

In [ ]:
# Definimos X e y
X_df = df_apartment.drop('price_usd_per_m2', axis=1)
y_df = df_apartment.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
scaler_ap = StandardScaler() #estandarizamos
X_train = scaler_ap.fit_transform(X_train)
X_test = scaler_ap.transform(X_test)

In [ ]:
#Asignamos los alphas
alphas = np.linspace(0.01, 1000, 300)
# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr_ap = lm.fit(X_train, y_train)
modelo_ridge_ap = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso_ap = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
pd.Series(y_test).describe()

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.scatter(y_test,y_test, s=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
#Distribución de la muestra
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

Imprimimos los coeficientes para cada modelo: 'APARTMENT' Intercept y coef de cada feature (fichur)

In [ ]:
####  Regresion Lineal  ####
print (lm.intercept_)
lm_coefs = list(zip(X_df.columns, lm.coef_))
lm_coefs =  sorted(lm_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de LM
lm_coefs = pd.DataFrame(lm_coefs, columns=('Features', 'LM_Coef'))
lm_coefs = lm_coefs.set_index('Features')
lm_coefs

In [ ]:
####  Ridge con CV  ####
print (lm_ridge_cv.intercept_)
ridge_coefs = list(zip(X_df.columns, lm_ridge_cv.coef_))
ridge_coefs = sorted(ridge_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Ridge
ridge_coefs = pd.DataFrame(ridge_coefs, columns=('Features','Ridge_Coef'))
ridge_coefs = ridge_coefs.set_index('Features')
ridge_coefs

In [ ]:
####  LASSO con CV  ####
print (lm_lasso_cv.intercept_)
lasso_coefs = list(zip(X_df.columns, lm_lasso_cv.coef_))
lasso_coefs = sorted(lasso_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Lasso
lasso_coefs = pd.DataFrame(lasso_coefs, columns=('Features','Lasso_Coef'))
lasso_coefs = lasso_coefs.set_index('Features')
lasso_coefs

In [ ]:
# Ploteamos los coeficientes de LM para cada feature en PH
sns.set(rc={'figure.figsize':(20,8)})
lm_coefs['LM_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Ridge para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
ridge_coefs['Ridge_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Lasso para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lasso_coefs['Lasso_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# Imprimimos un heatmap para evaluar las correlaciones de cada fichur y los coef calculados con el target
sns.set(rc={'figure.figsize':(35,4)})
sns.heatmap(pd.concat([X_df, pd.DataFrame(y_df)], axis=1).corr().iloc[[-1]], annot=True, cmap="Greens");

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Tipo de Propiedad: 'HOUSE'

In [ ]:
# definimos X e y
X_df = df_house.drop('price_usd_per_m2', axis=1)
y_df = df_house.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
scaler_ho = StandardScaler()
X_train = scaler_ho.fit_transform(X_train)
X_test = scaler_ho.transform(X_test)

In [ ]:
#Asignamos los alphas
alphas = np.linspace(0.1, 1000, 300)
# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr_ho = lm.fit(X_train, y_train)
modelo_ridge_ho = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso_ho = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.scatter(y_test,y_test, s=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
#Distribución de la muestra
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

In [ ]:
####  Regresion Lineal  ####
print (lm.intercept_)
lm_coefs = list(zip(X_df.columns, lm.coef_))
lm_coefs =  sorted(lm_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de LM
lm_coefs = pd.DataFrame(lm_coefs, columns=('Features', 'LM_Coef'))
lm_coefs = lm_coefs.set_index('Features')
lm_coefs

In [ ]:
####  Ridge con CV  ####
print (lm_ridge_cv.intercept_)
ridge_coefs = list(zip(X_df.columns, lm_ridge_cv.coef_))
ridge_coefs = sorted(ridge_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Ridge
ridge_coefs = pd.DataFrame(ridge_coefs, columns=('Features','Ridge_Coef'))
ridge_coefs = ridge_coefs.set_index('Features')
ridge_coefs

In [ ]:
####  LASSO con CV  ####
print (lm_lasso_cv.intercept_)
lasso_coefs = list(zip(X_df.columns, lm_lasso_cv.coef_))
lasso_coefs = sorted(lasso_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Lasso
lasso_coefs = pd.DataFrame(lasso_coefs, columns=('Features','Lasso_Coef'))
lasso_coefs = lasso_coefs.set_index('Features')
lasso_coefs

In [ ]:
# ploteamos los coeficientes de LM para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lm_coefs['LM_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Ridge para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
ridge_coefs['Ridge_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Lasso para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lasso_coefs['Lasso_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# Imprimimos un heatmap para evaluar las correlaciones de cada feature y los coeficientes calculados con el target
sns.set(rc={'figure.figsize':(35,4)})
sns.heatmap(pd.concat([X_df, pd.DataFrame(y_df)], axis=1).corr().iloc[[-1]], annot=True, cmap="Greens");

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Tipo de Propiedad: 'STORE'

In [ ]:
# definimos X e y
X_df = df_store.drop('price_usd_per_m2', axis=1)
y_df = df_store.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
scaler_st = StandardScaler()
X_train = scaler_st.fit_transform(X_train)
X_test = scaler_st.transform(X_test)

In [ ]:
#Asignamos los alphas
alphas = np.linspace(0.1, 1000, 300)
# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr_st = lm.fit(X_train, y_train)
modelo_ridge_st = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso_st = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
#Distribución de la muestra
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

Imprimimos los coeficientes para cada modelo: 'STORE' Intercept y coef de cada feature 

In [ ]:
####  Regresion Lineal  ####
print (lm.intercept_)
lm_coefs = list(zip(X_df.columns, lm.coef_))
lm_coefs =  sorted(lm_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de LM
lm_coefs = pd.DataFrame(lm_coefs, columns=('Features', 'LM_Coef'))
lm_coefs = lm_coefs.set_index('Features')
lm_coefs

In [ ]:
####  Ridge con CV  ####
print (lm_ridge_cv.intercept_)
ridge_coefs = list(zip(X_df.columns, lm_ridge_cv.coef_))
ridge_coefs = sorted(ridge_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Ridge
ridge_coefs = pd.DataFrame(ridge_coefs, columns=('Features','Ridge_Coef'))
ridge_coefs = ridge_coefs.set_index('Features')
ridge_coefs

In [ ]:
####  LASSO con CV  ####
print (lm_lasso_cv.intercept_)
lasso_coefs = list(zip(X_df.columns, lm_lasso_cv.coef_))
lasso_coefs = sorted(lasso_coefs, key=lambda x:x[1], reverse=True)
# armamos un df con los coeficientes de Lasso
lasso_coefs = pd.DataFrame(lasso_coefs, columns=('Features','Lasso_Coef'))
lasso_coefs = lasso_coefs.set_index('Features')
lasso_coefs

In [ ]:
# ploteamos los coeficientes de LM para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lm_coefs['LM_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Ridge para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
ridge_coefs['Ridge_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# ploteamos los coeficientes de Lasso para cada feature 
sns.set(rc={'figure.figsize':(20,8)})
lasso_coefs['Lasso_Coef'].plot(kind='bar', color=sns.color_palette(n_colors=1));

In [ ]:
# Imprimimos un heatmap para evaluar las correlaciones de cada fichur y los coef calculados con el target
sns.set(rc={'figure.figsize':(35,4)})
sns.heatmap(pd.concat([X_df, pd.DataFrame(y_df)], axis=1).corr().iloc[[-1]], annot=True, cmap="Greens");

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

   **---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en GBA](#Análisis-de-propiedades-en-GBA)
[Volver](#Índice)

### Análisis de propiedades en el Gran Buenos Aires (gba) ###


In [ ]:
df_gba = pd.read_csv('df_gba.csv', index_col=0)
display(df_gba.shape)#Nos fijamos con cuantas filas y columnas arrancamos
display(df_gba.sample(5))#elegimos un sample de 10 para que el muestreo sea aleatorio
df_gba.isnull().sum()# Visualización de las columnas con valores null

In [ ]:
# Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2 y rooms.
df_gba = df_gba[(df_gba.surface_total_in_m2.notnull())
                  & (df_gba.price_usd_per_m2.notnull())
                  & (df_gba.rooms.notnull())
                  & (df_gba.localidad.notnull())]
df_gba.isnull().sum()

In [ ]:
#Generamos las dummies para barrios (usando la columna de localidad que es la que contiene dicha información)
gba_dummies = pd.get_dummies(df_gba.localidad, prefix='barrio')
gba_dummies.columns
df_gba = pd.concat([df_gba, gba_dummies], axis=1) #concatenamos las dummies generadas con el df de gba
df_gba.head()

In [ ]:
# #Eliminamos las columnas que no van a ser utilizadas para la regresión. 
df_gba.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
display(df_gba.sample(5))
display(df_gba.shape)

Después de esta limpieza, para el analisis de regresión vamos a tener 15212 filas y  51 columnas

Analisis de regresión

In [ ]:
# definimos X e y
X_gba = df_gba.drop('price_usd_per_m2', axis=1)
y_gba = df_gba.price_usd_per_m2
display(X_gba.shape)
display(y_gba.shape)

In [ ]:
# separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_gba, y_gba, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#asignamos los alphas
alphas = np.linspace(0.1, 10000, 300)

# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

--------------------------------------------------------------------------------------------------

Vamos a dividir por región del GBA para ver si estos resultados mejoran

In [ ]:
def preparar_df_gba( df_param ):
    #generamos las dummies para barrios (usando la columna de localidad que es la que contiene dicha información)
    gba_dummies = pd.get_dummies(df_param.localidad, prefix='barrio')
    gba_dummies.columns
    df_retorno = pd.concat([df_param, gba_dummies], axis=1) #concatenamos las dummies generadas con el df de gba

    # #Eliminamos las columnas que no van a ser utilizadas para la regresión. Saco del df el contador de amenities
    df_retorno.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
                 'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
                 'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
                  inplace=True, axis=1)
    display(df_retorno.sample(5))
    display(df_retorno.shape)
    return df_retorno

In [ ]:
def outliers( df, criterio, p_z_score ):
    criterios = df[ criterio ].unique()
    for x in criterios:
           mean_1 = np.mean( df.price_usd_per_m2[ ( df[ criterio ] ==x ) ] )
           std_1 = np.std( df.price_usd_per_m2[ ( df[ criterio ] ==x ) ] )
           mean_2 = np.mean( df.surface_total_in_m2[ ( df[ criterio ] == x ) ] )
           std_2 = np.std( df.surface_total_in_m2[ ( df[ criterio ] == x ) ] )

           for index2, y in df[ ( df[ criterio ] == x ) ].iterrows():
                   if( std_1 != 0 ):
                       z_score = ( y.price_usd_per_m2 - mean_1 )/std_1
                       if np.abs( z_score ) > p_z_score:
                           df.loc[ index2, 'outlier' ] = "YES"
                   if( std_2 != 0 ):
                       z_score2= ( y.surface_total_in_m2 - mean_2 )/std_2
                       if np.abs( z_score2 ) > p_z_score:
                           df.loc[ index2, 'outlier' ]="YES"

    df = df[(df.outlier!='YES')]
    df.drop(['outlier'],inplace=True, axis=1)
    return df

In [ ]:
def df_fit_transform( df, y_val ):
    # Zona Norte
    # definimos X e y
    X_df = df.drop(y_val, axis=1)
    y_df = df[ y_val ]
    display(X_df.shape)
    display(y_df.shape)

    # separamos los datos para train y test
    X_train_no_norm, X_test_no_norm, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)

    print(X_train_no_norm.shape, y_train.shape)
    print(X_test_no_norm.shape, y_test.shape)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_no_norm) 
    X_test = scaler.transform(X_test_no_norm)

    #asignamos los alphas
    alphas = np.linspace(0.1, 1000, 300)

    # Definimos kf para utilizar kFold como estrategia de cross validation.
    kf = KFold(n_splits=5, shuffle=True, random_state=68)

    #Instanciamos los modelos 
    lm = LinearRegression()
    lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
    lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

    # Entrenamos los modelos
    modelo_lr = lm.fit(X_train, y_train)
    modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
    modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

    #Imprimimos el alpha que eligio para cada modelo
    print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
          'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

    # Calculamos el RMSE

    y_pred_tr_lm = lm.predict(X_train)
    y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
    y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

    rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

    y_pred_lm = lm.predict(X_test)
    y_pred_ridge = lm_ridge_cv.predict(X_test)
    y_pred_lasso = lm_lasso_cv.predict(X_test)


    scores_eval = dict()
    scores_eval.update({"Score Train":{ "linear":lm.score(X_train, y_train),"ridge":lm_ridge_cv.score(X_train, y_train),"lasso":lm_lasso_cv.score(X_train, y_train)}})
    scores_eval.update({"RMSE Train":{ "linear":rmse(y_train,y_pred_tr_lm),"ridge":rmse(y_train,y_pred_tr_ridge),"lasso":rmse(y_train,y_pred_tr_lasso)}})
    scores_eval.update({"R2 Test":{ "linear":r2_score(y_test, y_pred_lm),"ridge":r2_score(y_test, y_pred_ridge),"lasso":r2_score(y_test, y_pred_lasso)}})
    scores_eval.update({"RMSE Test":{ "linear":rmse(y_test, y_pred_lm),"ridge":rmse(y_test, y_pred_ridge),"lasso":rmse(y_test, y_pred_lasso)}})

    data_eval = dict()
    
    data_eval.update({"intercept":{ "linear":lm.intercept_,"ridge":lm_ridge_cv.intercept_,"lasso":lm_lasso_cv.intercept_}})
    for idx, col in enumerate(X_df.columns):
        data_eval.update({col:{ "linear":lm.coef_[idx],"ridge":lm_ridge_cv.coef_[idx],"lasso":lm_lasso_cv.coef_[idx]}})

    ##### Grafico de cada variable sobre el y_target de la prectica train test split###
    sns.set(rc={'figure.figsize':(35,4)})
    sns.heatmap(pd.concat([X_df, pd.DataFrame(y_df)], axis=1).corr().iloc[[-1]], annot=True);
    ### aca no se que significan los parametros iloc-1 y annot= True (anota los valores dentro de las celdas)

    display(pd.DataFrame(data_eval))
    display(pd.DataFrame(scores_eval))

    retorno = dict()
    retorno = { "linear": lm, "ridge": lm_ridge_cv, "lasso": lm_lasso_cv }

    return retorno

In [ ]:
df_gba = pd.read_csv('df_gba.csv', index_col=0)
df_gba = df_gba[(df_gba.surface_total_in_m2.notnull())
                  & (df_gba.price_usd_per_m2.notnull())
                  & (df_gba.rooms.notnull())
                  & (df_gba.provincia.notnull())]
df_gba.isnull().sum()

In [ ]:
df_gba = outliers( df = df_gba, criterio = 'provincia', p_z_score = 3 )

In [ ]:
db_gba_norte = df_gba[( df_gba.provincia == 'Bs.As. G.B.A. Zona Norte' )]
db_gba_sur = df_gba[( df_gba.provincia == 'Bs.As. G.B.A. Zona Sur' )]
db_gba_oeste = df_gba[( df_gba.provincia == 'Bs.As. G.B.A. Zona Oeste' )]

In [ ]:
db_gba_norte_dummies = preparar_df_gba( db_gba_norte )

db_gba_sur_dummies = preparar_df_gba( db_gba_sur )

db_gba_oeste_dummies = preparar_df_gba( db_gba_oeste )

Después de esta limpieza y separación, para el analisis de regresión vamos a tener:

Zona Norte: 8139 filas y 29 columnas

Zona Sur: 3153 filas 32 columnas

Zona Oeste: 3485 filas y 28 columnas

Analisis de regresión

In [ ]:
models_gba_norte = df_fit_transform( df = db_gba_norte_dummies, y_val = 'price_usd_per_m2' )

In [ ]:
models_gba_sur = df_fit_transform( df = db_gba_sur_dummies, y_val = 'price_usd_per_m2' )

In [ ]:
models_gba_oeste = df_fit_transform( df = db_gba_oeste_dummies, y_val = 'price_usd_per_m2' )

In [ ]:
models_gba_sur['linear'].intercept_

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Provincia de Buenos Aires](#Análisis-de-Propiedades-en-Provincia-de-Buenos-Aires)
[Volver](#Índice)

### Análisis de propiedades en Bs As provincia ###


In [ ]:
df_bs_as_prov = pd.read_csv('df_bs_as_prov.csv', index_col=0)
display(df_bs_as_prov.shape)#Nos fijamos con cuantas filas y columnas arrancamos
display(df_bs_as_prov.sample(10))#elegimos un sample de 10 para que el muestreo sea aleatorio
df_bs_as_prov.isnull().sum() # Visualización de las columnas con valores null

In [ ]:
df=df_bs_as_prov  
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 2: 
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_bs_as_prov=df[(df.outlier!='YES')]

In [ ]:
df_bs_as_prov.drop(['outlier'], inplace=True, axis=1)

In [ ]:
df_bs_as_prov['price_usd_per_m2'].describe()  

In [ ]:
#Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2, rooms y localidad.
df_bs_as_prov = df_bs_as_prov[(df_bs_as_prov.surface_total_in_m2.notnull())\
                  & (df_bs_as_prov.price_usd_per_m2.notnull())\
                  & (df_bs_as_prov.rooms.notnull())
                  & (df_bs_as_prov.localidad.notnull())]
df_bs_as_prov.isnull().sum()

In [ ]:
#Generamos las dummies para barrios (usando la columna de localidad que es la que contiene dicha información)
bs_as_prov_dummies = pd.get_dummies(df_bs_as_prov.localidad, prefix='localidad')
bs_as_prov_dummies.columns
df_bs_as_prov = pd.concat([df_bs_as_prov, bs_as_prov_dummies], axis=1) #concatenamos las dummies generadas con el df de gba
df_bs_as_prov.head()

In [ ]:
display(df_bs_as_prov.shape)
df_amen_bs_as_prov = df_bs_as_prov['amenities'].value_counts(normalize=True, dropna=False)
df_amen2_bs_as_prov =  df_bs_as_prov['amenities'].value_counts()
display(pd.concat([df_amen_bs_as_prov, df_amen2_bs_as_prov], axis=1))
plot = (100 * df_bs_as_prov['amenities'].value_counts(normalize = True)).plot(kind='bar', title='amenities %', figsize=(5,5))

Observamos el contador de las amenities en Bs As provincia. Decidimos incluirlas como feature.

In [ ]:
#Eliminamos las columnas que no van a ser utilizadas para la regresión. 
df_bs_as_prov.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
display(df_bs_as_prov.head())
display(df_bs_as_prov.shape)
display(df_bs_as_prov['price_usd_per_m2'].describe())

In [ ]:
quien_es1 = df_bs_as_prov[df_bs_as_prov['price_usd_per_m2']>=8000.00]
quien_es1 #eliminamos la propiedad ya que es de 20m2 ytiene 7 habitaciones ya creemos que no es posible

In [ ]:
df_bs_as_prov.drop(46162, axis=0, inplace=True)
display(df_bs_as_prov.shape)

Después de esta limpieza, para el analisis de regresión vamos a tener 5942 filas y 89 columnas

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_bs_as_prov = df_bs_as_prov.drop('price_usd_per_m2', axis=1)
y_bs_as_prov = df_bs_as_prov.price_usd_per_m2
display(X_bs_as_prov.shape)#MODIFIQUE Q ESTABA MAL
display(y_bs_as_prov.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_bs_as_prov, y_bs_as_prov, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas y definimos kf para utilizar kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 1000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO")
plt.ylabel("Valores reales precio en USD por m2")
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Patagonia](#Análisis-de-propiedades-en-Patagonia)
[Volver](#Índice)

### Análisis de propiedades en Patagonia ###


In [ ]:
df_patagonia = pd.read_csv('df_patagonia.csv', index_col=0)
display(df_patagonia.shape)#Nos fijamos con cuantas filas y columnas arrancamos
display(df_patagonia.sample(5))#elegimos un sample de 10 para que el muestreo sea aleatorio
display(df_patagonia.isnull().sum())
display(df_patagonia['price_usd_per_m2'].describe())# Visualización de las columnas con valores null

In [ ]:
df=df_patagonia 
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3: # cualquier punto fuera de 3 desviaciones estandard fueron considerados outlier.
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_patagonia=df[(df.outlier!='YES')]

In [ ]:
df_patagonia.drop(['outlier'], inplace=True, axis=1)

In [ ]:
df_patagonia['price_usd_per_m2'].describe()  #remoción correcta de outliers

In [ ]:
quienes2 = df_patagonia[df_patagonia['price_usd_per_m2']==10500.00]
quienes2

In [ ]:
#Como Provincia es la columna que vamos a utilizar para las dummies tiene 0 nulls, transformo dicha columna a variables dummies
patagonia_dummies = pd.get_dummies(df_patagonia.provincia, prefix='Provincia_de')
patagonia_dummies.columns
df_pat = pd.concat([df_patagonia, patagonia_dummies], axis=1) #concatenamos las dummies generadas con el df de patagonia
df_pat.head()

In [ ]:
df_amen_pat = df_pat['amenities'].value_counts(normalize=True, dropna=False)
df_amen2_pat =  df_pat['amenities'].value_counts()
display(pd.concat([df_amen_pat, df_amen2_pat], axis=1))
plot = (100 * df_pat['amenities'].value_counts(normalize = True)).plot(kind='bar', title='amenities %', figsize=(5,5))

In [ ]:
#Eliminamos las columnas que no van a ser utilizadas para la regresión. Saco del df el contador de amenities
df_pat.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
df_pat.head()

In [ ]:
#Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2,rooms.
df_patagonia = df_pat[(df_pat.surface_total_in_m2.notnull())\
                  & (df_pat.price_usd_per_m2.notnull())\
                  & (df_pat.rooms.notnull())]
display(df_patagonia.isnull().sum())
display(df_patagonia.shape)
display(df_patagonia['price_usd_per_m2'].describe())

Después de esta limpieza, para el analisis de regresión vamos a tener 158 filas y 23 columnas

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando heatmap

plt.rcParams['font.size'] = 7
plt.rcParams['figure.figsize'] = (30, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_patagonia.corr(), annot=True, fmt='.2f', cmap="BuPu");

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_patagonia = df_patagonia.drop('price_usd_per_m2', axis=1)
y_patagonia = df_patagonia.price_usd_per_m2
display(X_patagonia.shape)
display(y_patagonia.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_patagonia, y_patagonia, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas y kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 1000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

In [ ]:
#dropeando refacción
df_patagonia.drop(['refaccion'], axis=1, inplace=True)

In [ ]:
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (30, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_patagonia.corr(), annot=True, fmt='.2f', cmap="BuPu");

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Región Pampeana](#Análisis-de-propiedades-en-Región-Pampeana)
[Volver](#Índice)

### Análisis de propiedades en la región pampeana ###
**Nota: No agrupamos a Bs As en este DataFrame**

In [ ]:
df_pampeana = pd.read_csv('df_pampeana.csv', index_col=0)
display(df_pampeana.shape)#Nos fijamos con cuantas filas y columnas arrancamos
display(df_pampeana.sample(5))#elegimos un sample  para que el muestreo sea aleatorio
display(df_pampeana.isnull().sum()) # Visualización de las columnas con valores null
display(df_pampeana['price_usd_per_m2'].describe())

Encontramos un super outlier. Empezamos a pensar que hay valores que no estaban en usd sino en pesos y no supimos verlo en el primer desafio. 

In [ ]:
quienes3 = df_pampeana[df_pampeana['price_usd_per_m2']==600000.00] #como tiene rooms en null cuando hagamos el filtro por nulls (que es antes de la regresion) va a desaparecer
quienes3

In [ ]:
df=df_pampeana  
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 2: 
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_pampeana=df[(df.outlier!='YES')]

In [ ]:
df_pampeana.drop(['outlier'], inplace=True, axis=1)

In [ ]:
##Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2,rooms y provincia.
df_pampeana = df_pampeana[(df_pampeana.surface_total_in_m2.notnull())\
                  & (df_pampeana.price_usd_per_m2.notnull())\
                  & (df_pampeana.rooms.notnull())\
                  & (df_pampeana.provincia.notnull())]
df_pampeana.isnull().sum()

In [ ]:
df_pampeana['price_usd_per_m2'].describe()  #remoción correcta de outliers

In [ ]:
quien_es4 = df_pampeana[df_pampeana['price_usd_per_m2']==6000.00] #
quien_es4

5 rooms nunca pueden ocupar 20m2. Procedemos a eliminar esta fila

In [ ]:
df_pampeana.drop(40075, axis=0, inplace=True)

In [ ]:
#Provincia es la columna que vamos a utilizar para las dummies tiene 0 nulls, transformo dicha columna a variables dummies
pampeana_dummies = pd.get_dummies(df_pampeana.provincia, prefix='Provincia_de')
pampeana_dummies.columns
df_pam = pd.concat([df_pampeana, pampeana_dummies], axis=1) #concatenamos las dummies generadas con el df de patagonia
df_pam.head()

In [ ]:
df_amen_pam = df_pam['amenities'].value_counts(normalize=True, dropna=False)
df_amen2_pam =  df_pam['amenities'].value_counts()
display(pd.concat([df_amen_pam, df_amen2_pam], axis=1))
plot = (100 * df_pam['amenities'].value_counts(normalize = True)).plot(kind='bar', title='amenities %', figsize=(5,5))

In [ ]:
#Eliminamos las columnas que no van a ser utilizadas para la regresión
df_pam.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
df_pam.shape

In [ ]:
df_pam['price_usd_per_m2'].describe() 

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando heatmap

plt.rcParams['font.size'] = 7
plt.rcParams['figure.figsize'] = (30, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_pam.corr(), annot=True, fmt='.2f', cmap="BuPu");

Después de esta limpieza, para el analisis de regresión vamos a tener 2697 filas y 22 columnas

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_pampeana = df_pam.drop('price_usd_per_m2', axis=1)
y_pampeana = df_pam.price_usd_per_m2
display(X_pampeana.shape)
display(y_pampeana.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_pampeana, y_pampeana, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#asignamos los alphas y definimos kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 100, 300)

kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()


In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.scatter(y_test,y_test, s=10)
plt.xlabel("Predicciones usando Regresion Lineal", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

**----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Noroeste](#Análisis-de-propiedades-en-Noroeste)
[Volver](#Índice)

### Análisis de propiedades en la región noroeste ###

In [ ]:
df_noroeste = pd.read_csv('df_noroeste.csv', index_col=0)
display(df_noroeste.shape)#Nos fijamos con cuantas filas y columnas arrancamos
display(df_noroeste.sample(10))#elegimos un sample  para que el muestreo sea aleatorio
display(df_noroeste.isnull().sum())
display(df_noroeste['price_usd_per_m2'].describe())# Visualización de las columnas con valores null

In [ ]:
quien_es5 = df_noroeste[df_noroeste['price_usd_per_m2']>=5400] #Esta propiedad tiene rooms en nan (y ya hicimos el regex en titulo y descripcion y sabemos que no hay info). Va a ser eliminada mas abajo con los nulls
quien_es5

In [ ]:
df=df_noroeste  
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3: # cualquier punto fuera de 3 desviaciones estandard son considerados outlier.
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_noroeste=df[(df.outlier!='YES')]

In [ ]:
df_noroeste.drop(['outlier'], inplace=True, axis=1)

In [ ]:
#Provincia es la columna que vamos a utilizar para las dummies tiene 0 nulls, transformo dicha columna a variables dummies
noroeste_dummies = pd.get_dummies(df_noroeste.provincia, prefix='Provincia_de')
noroeste_dummies.columns
df_noroeste = pd.concat([df_noroeste, noroeste_dummies], axis=1) #concatenamos las dummies generadas con el df de patagonia
df_noroeste.head()

In [ ]:
##Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2 y rooms 
df_noroeste = df_noroeste[(df_noroeste.surface_total_in_m2.notnull())\
                  & (df_noroeste.price_usd_per_m2.notnull())\
                  & (df_noroeste.rooms.notnull())]
df_noroeste.isnull().sum()

In [ ]:
#Eliminamos las columnas que no van a ser utilizadas para la regresión
df_noroeste.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
display(df_noroeste.head())
display(df_noroeste['price_usd_per_m2'].describe())
display(df_noroeste.shape)

Después de esta limpieza, quedan muy pocos datos y para el analisis de regresión vamos a tener 30 filas y 24 columnas

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando heatmap
plt.rcParams['font.size'] = 7
plt.rcParams['figure.figsize'] = (30, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_noroeste.corr(), annot=True, fmt='.2f', cmap="BuPu");

Eliminamos refacción, PH, store, barrio cerrado, vigilancia y las provincias de Jujuy y La Rioja

In [ ]:
#dropeando las variables irrelevantes
df_noroeste.drop(['refaccion', 'store', 'PH', 'vigilancia', 'barrio_cerrado', 'Provincia_de_Jujuy',
                  'Provincia_de_La Rioja'], axis=1, inplace=True)



In [ ]:
# visualizamos la matriz de correlación en Seaborn usando heatmap
plt.rcParams['font.size'] = 7
plt.rcParams['figure.figsize'] = (30, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_noroeste.corr(), annot=True, fmt='.2f', cmap="BuPu");

**Analisis de regresión**

In [ ]:
# Definimos X e y
X_noroeste = df_noroeste.drop('price_usd_per_m2', axis=1)
y_noroeste = df_noroeste.price_usd_per_m2
display(X_noroeste.shape)
display(y_noroeste.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_noroeste, y_noroeste, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#asignamos los alphas
alphas = np.linspace(0.1, 100, 300)

# Definimos kf para utilizar kFold como estrategia de cross validation.
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.xlabel("Predicciones usando Regresion Lineal", size=12)
plt.ylabel("Valores reales precio en USD por m2",size=12)
plt.show()


In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Nordeste](#Análisis-de-propiedades-en-Nordeste)
[Volver](#Índice)

### Análisis de propiedades en la región nordeste ###


In [ ]:
df_nordeste = pd.read_csv('df_nordeste.csv', index_col=0)
display(df_nordeste.shape) #Nos fijamos con cuantas filas y columnas arrancamos
display(df_nordeste.sample(10))#elegimos un sample  para que el muestreo sea aleatorio
display(df_nordeste.isnull().sum())
display(df_nordeste['price_usd_per_m2'].describe())# Visualización de las columnas con valores null

In [ ]:
df=df_nordeste  
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3: # cualquier punto fuera de 3 desviaciones estandard son considerados outlier.
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_nordeste=df[(df.outlier!='YES')]

In [ ]:
df_nordeste.drop(['outlier'], inplace=True, axis=1)

In [ ]:
df_nordeste['price_usd_per_m2'].describe()  #remoción correcta de outliers

In [ ]:
quien_es5 = df_nordeste[df_nordeste['price_usd_per_m2']>=7000] #Es posible que esta propiedad este bien cotizada en Puerto Iguazu que es una zona turistica
quien_es5

In [ ]:
#como Provincia es la columna que vamos a utilizar para las dummies tiene 0 nulls, transformamos dicha columna a variables dummies
nordeste_dummies = pd.get_dummies(df_nordeste.provincia, prefix='Provincia')
nordeste_dummies.columns
df_nord = pd.concat([df_nordeste, nordeste_dummies], axis=1) #concatenamos las dummies generadas con el df de la region Nordeste
df_nord.head()

In [ ]:
#Eliminamos las columnas que no van a ser utilizadas para la regresión
df_nord.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 
             'price_aprox_usd', 'pileta','parrilla', 'gimnasio', 'sauna', 'laundry', 
             'playroom', 'sum', 'sala_reuniones','restaurant', 'quincho'],
              inplace=True, axis=1)
df_nord.head()

In [ ]:
df_amen_nord = df_nordeste['amenities'].value_counts(normalize=True, dropna=False)
df_amen2_nord =  df_nordeste['amenities'].value_counts()
display(pd.concat([df_amen_nord, df_amen2_nord ], axis=1))
plot = (100 * df_nord['amenities'].value_counts(normalize = True)).plot(kind='bar', title='amenities %', figsize=(5,5))

In [ ]:
#Eliminamos las filas con valores nan para surface_total_in_m2,price_usd_per_m2,rooms.
df_nordeste = df_nord[(df_nord.surface_total_in_m2.notnull())\
                  & (df_nord.price_usd_per_m2.notnull())\
                  & (df_nord.rooms.notnull())]
display(df_nordeste.isnull().sum())
display(df_nordeste.shape)

###### Después de esta limpieza, para el analisis de regresión vamos a tener 268 filas y 23 columnas

In [ ]:
# Visualizamos la matriz de correlación en Seaborn usando heatmap
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (25, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_nordeste.corr(), annot=True, fmt='.2f', cmap="BuPu");

Analisis de regresión

In [ ]:
# Definimos X e y
X_nordeste = df_nordeste.drop('price_usd_per_m2', axis=1)
y_nordeste = df_nordeste.price_usd_per_m2
display(X_nordeste.shape)
display(y_nordeste.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_nordeste, y_nordeste, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas y definimos kf para utilizar kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 1000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal
plt.figure(figsize=(5, 5))
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.scatter(y_test,y_test, s=10)
plt.xlabel("Predicciones usando Regresion Lineal", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

In [ ]:
#dropeando las variables irrelevantes
df_nordeste.drop(['refaccion', 'store', 'Provincia_Formosa'], axis=1, inplace=True)

   **-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# [Análisis de Propiedades en Cuyo](#Análisis-de-propiedades-en-Cuyo)
[Volver](#Índice)

###  Análisis de propiedades en la región de Cuyo ###

In [ ]:
df_cuyo = pd.read_csv('df_cuyo.csv', index_col=0)
display(df_cuyo.shape) #Nos fijamos con cuantas filas y columnas arrancamos
display(df_cuyo.sample(10))#selección de 10 ejemplares al azar
display(df_cuyo.isnull().sum())
display(df_cuyo['price_usd_per_m2'].describe())# Visualización de las columnas con valores null

In [ ]:
df=df_cuyo  
localidades=df.localidad.unique()
for x in localidades:
       mean_1 = np.mean(df.price_usd_per_m2[df.localidad==x])
       std_1 =np.std(df.price_usd_per_m2[df.localidad==x])
       mean_2 = np.mean(df.surface_total_in_m2[df.localidad==x])
       std_2 =np.std(df.surface_total_in_m2[df.localidad==x])

       for index2, y in df[df.localidad==x].iterrows():
               if(std_1!=0):
                   z_score= (y.price_usd_per_m2 - mean_1)/std_1
                   if np.abs(z_score) > 3: # cualquier punto fuera de 3 desviaciones estandard son considerados outlier.
                       df.loc[index2,'outlier']="YES"
               if(std_2!=0):
                   z_score2= (y.surface_total_in_m2 - mean_2)/std_2
                   if np.abs(z_score2) > 3:
                       df.loc[index2,'outlier']="YES"

In [ ]:
df_cuyo=df[(df.outlier!='YES')]

In [ ]:
df_cuyo.drop(['outlier'], inplace=True, axis=1)

In [ ]:
df_cuyo['price_usd_per_m2'].describe()  #remoción correcta de outliers

In [ ]:
cuyo_dummies = pd.get_dummies(df_cuyo.provincia, prefix='Provincia')
df_cuyo = pd.concat([df_cuyo, cuyo_dummies], axis=1) 
display(df_cuyo.shape)
display(df_cuyo.sample(5))

In [ ]:
df_cuyo = df_cuyo[(df_cuyo.surface_total_in_m2.notnull())\
                  & (df_cuyo.price_usd_per_m2.notnull())\
                  & (df_cuyo.rooms.notnull())]
display(df_cuyo.shape)
display(df_cuyo.isnull().sum()) 

Hay pocas propiedades en la región de Cuyo después de la limpieza

In [ ]:
df_amen_cuyo = df_cuyo['amenities'].value_counts(normalize=True, dropna=False)
df_amen2_cuyo =  df_cuyo['amenities'].value_counts()
display(pd.concat([df_amen_cuyo, df_amen2_cuyo], axis=1))
plot = (100 * df_cuyo['amenities'].value_counts(normalize = True)).plot(kind='bar', title='amenities %', figsize=(5, 5))

In [ ]:
#Eliminamos las colamumnas que no van a ser utilizadas para la regresión
df_cuyo.drop(['lat-lon', 'lat', 'lon', 'localidad', 'barrio', 'provincia', 'geonames_id', 'price_aprox_usd',
            'pileta', 'parrilla', 'gimnasio', 'sauna', 'laundry', 'playroom', 'sum', 'sala_reuniones', 
            'restaurant', 'quincho',], inplace=True, axis=1)
display(df_cuyo.shape)
display(df_cuyo.head())

Después de esta limpieza, para el analisis de regresión vamos a tener 44 filas y 22 columnas

In [ ]:
# Visualizamos la matriz de correlación en Seaborn usando heatmap
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (20, 10)
sns.set(font_scale=1.3)
sns.heatmap(df_cuyo.corr(), annot=True, fmt='.2f', cmap="BuPu");

A estrenar, en refacción o normal no parecen estar aportando datos a la correlación. Tampoco vigilancia, PH y la provincia de San Juan.

In [ ]:
#dropeando las variables irrelevantes
df_cuyo.drop(['vigilancia', 'estrenar','refaccion', 'Provincia_San Juan', 'normal', 'PH'], axis=1, inplace=True)

plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (25, 15)
sns.set(font_scale=1.7)
sns.heatmap(df_cuyo.corr(), annot=True, fmt='.2f', cmap="BuPu");

In [ ]:
# Definimos X e y
X_df = df_cuyo.drop('price_usd_per_m2', axis=1)
y_df = df_cuyo.price_usd_per_m2
display(X_df.shape)
display(y_df.shape)

In [ ]:
# Separamos los datos para train y test
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
#Estandarizamos los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Asignamos los alphas y kFold como estrategia de cross validation.
alphas = np.linspace(0.1, 1000, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=68)

In [ ]:
#Instanciamos los modelos 
lm = LinearRegression()
lm_ridge_cv= RidgeCV(alphas=alphas, cv=kf, normalize=False)
lm_lasso_cv = LassoCV(alphas=alphas, cv=kf, normalize=False)

In [ ]:
# Entrenamos los modelos
modelo_lr = lm.fit(X_train, y_train)
modelo_ridge = lm_ridge_cv.fit(X_train, y_train)
modelo_lasso = lm_lasso_cv.fit(X_train, y_train)

In [ ]:
#Imprimimos el alpha que eligio para cada modelo
print('El 𝛼 estimado en Ridge: %.4f'% lm_ridge_cv.alpha_,'\n'
      'El 𝛼 estimado en LASSO: %.4f'% lm_lasso_cv.alpha_,'\n')

In [ ]:
# Imprimimos el Score de entrenamiento de cada modelo, si dan muy altos es porque estan overfiteados

print(" Score Train Lineal: %.4f\n" % lm.score(X_train, y_train),
      "Score Train Ridge : %.4f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.4f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE

y_pred_tr_lm = lm.predict(X_train)
y_pred_tr_ridge = lm_ridge_cv.predict(X_train)
y_pred_tr_lasso = lm_lasso_cv.predict(X_train)

rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))

print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_tr_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_tr_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_tr_lasso))

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

y_pred_lm = lm.predict(X_test)
y_pred_ridge = lm_ridge_cv.predict(X_test)
y_pred_lasso = lm_lasso_cv.predict(X_test)

# Testeo final del modelo sobre Test Set

print(" Test R2 lineal= %.4f\n" % r2_score(y_test, y_pred_lm),
      "Test R2 Ridge = %.4f\n" %  r2_score(y_test, y_pred_ridge),
      "Test R2 Lasso = %.4f" %  r2_score(y_test, y_pred_lasso))
print()

print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lm),
      "Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridge),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lasso))

In [ ]:
# Graficamos los resultados para el modelo Regresion Lineal

plt.figure(figsize=(5, 5))
plt.scatter(y_pred_lm, y_test, s=30, c='r', marker='.', zorder=10)
plt.scatter(y_test,y_test, s=10)
plt.xlabel("Predicciones usando Regresion Lineal", size = 12)
plt.ylabel("Valores reales precio en USD por m2", size = 12)
plt.show()

In [ ]:
# Graficamos los resultados para el modelo RIDGE
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=10)
plt.scatter(y_pred_ridge, y_test, s=30, c='g', marker='.', zorder=10)
plt.xlabel("Predicciones usando RIDGE", size = 12)
plt.ylabel("Valores reales precio en USD por m2", size = 12)
plt.show()

In [ ]:
pd.Series(y_test).hist(bins=100, figsize=(20, 5));

In [ ]:
# Graficamos los resultados para el modelo LASSO
plt.figure(figsize=(5, 5))
plt.scatter(y_test,y_test, s=5)
plt.scatter(y_pred_lasso, y_test, s=30, c='purple', marker='.', zorder=10)
plt.xlabel("Predicciones usando LASSO", size=12)
plt.ylabel("Valores reales precio en USD por m2", size=12)
plt.show()

# [Análisis de Sample en CABA](#Análisis-de-Sample-en-CABA)
[Volver](#Índice)

Importamos el Dataframe de los 100 Samples de CABA

In [ ]:
df_caba_sample = pd.read_csv('caba_sample.csv', index_col=0)

Armamos un Dataframe por cada tipo de Propiedad

In [ ]:
df_ph_sample = df_caba_sample[(df_caba_sample.PH== 1)]
df_apartment_sample = df_caba_sample[(df_caba_sample.apartment== 1)]
df_store_sample = df_caba_sample[(df_caba_sample.store== 1)]
df_house_sample = df_caba_sample[(df_caba_sample.house== 1)]

df_ph_sample.drop(['apartment', 'store', 'PH', 'house'], inplace=True, axis=1)
df_apartment_sample.drop(['apartment', 'store', 'PH', 'house'], inplace=True, axis=1)
df_store_sample.drop(['apartment', 'store', 'PH', 'house'], inplace=True, axis=1)
df_house_sample.drop(['apartment', 'store', 'PH', 'house'], inplace=True, axis=1)

Definimos la funcion a utilizar para predecir

In [ ]:
def prediccion_sample_caba( df, scaler, linear_model, ridge_model, lasso_model ):
    # definimos X e y
    X_df = df.drop('price_usd_per_m2', axis=1)
    y_df = df.price_usd_per_m2

    # Escalamos los valores con el escalador fiteado al momento de armar el Modelo, para cada tipo de propiedad,
    # ya que los modelos fueron entrenados con los valores escalados.
    X_df = scaler.transform(X_df)

    # Predecimos con modelo lineal
    y_pred_lm = linear_model.predict(X_df)
    
    # Predecimos con Ridge
    y_pred_ridge = ridge_model.predict(X_df)
    
    # Predecimos con Lasso
    y_pred_lasso = lasso_model.predict(X_df)

    ### Visualizamos y comparamos los valores de cada predictor con el real del dataframe original ###
    lista = list()
    for idx, val in enumerate( X_df ):
      lista.append( { "real": y_df.iloc[ idx ], "linear": y_pred_lm[ idx ], "ridge":y_pred_ridge[ idx ], "lasso":y_pred_lasso[ idx ] } )

    df_retorno = pd.DataFrame(lista)
    display(df_retorno)
    return df_retorno
    #############################

Predecimos el Sample de PH

In [ ]:
pred_ph = prediccion_sample_caba( df = df_ph_sample, scaler = scaler_ph, linear_model = modelo_lr_ph, ridge_model = modelo_ridge_ph, lasso_model = modelo_lasso_ph)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(pred_ph.real,pred_ph.real, label='Real')
plt.scatter(pred_ph.linear, pred_ph.real, s=100, c='r', marker='o', zorder=10, label='Lineal')
plt.scatter(pred_ph.ridge, pred_ph.real, s=100, c='g', marker='+', zorder=10, label='Ridge')
plt.scatter(pred_ph.lasso, pred_ph.real, s=100, c='purple', marker='v', zorder=10, label='Lasso')
plt.xlabel("Predicciones ", size=12)
plt.ylabel("Valores reales PH precio en USD por m2", size=12)
plt.legend()

plt.show()

Predecimos el Sample de Apartment

In [ ]:
pred_ap = prediccion_sample_caba( df = df_apartment_sample, scaler = scaler_ap, linear_model = modelo_lr_ap, ridge_model = modelo_ridge_ap, lasso_model = modelo_lasso_ap)

In [ ]:
pred_ap = pred_ap[ (pred_ap.real < 21085) ]
plt.figure(figsize=(5, 5))
plt.plot(pred_ap.real,pred_ap.real, label='Real')
plt.scatter(pred_ap.linear, pred_ap.real, s=100, c='r', marker='o', zorder=10, alpha=0.3, label='Lineal')
plt.scatter(pred_ap.ridge, pred_ap.real, s=200, c='g', marker='+', zorder=10, label='Ridge')
plt.scatter(pred_ap.lasso, pred_ap.real, s=100, c='purple', marker='v', zorder=10, alpha=0.3, label='Lasso')
plt.xlabel("Predicciones ", size=12)
plt.ylabel("Valores reales Apartment precio en USD por m2", size=12)

plt.legend()
plt.show()

Predecimos el Sample de House

In [ ]:
pred_ho = prediccion_sample_caba( df = df_house_sample, scaler = scaler_ho, linear_model = modelo_lr_ho, ridge_model = modelo_ridge_ho, lasso_model = modelo_lasso_ho)

Predecimos el Sample de House



In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(pred_ho.real,pred_ho.real, label='Real')
plt.scatter(pred_ho.linear, pred_ho.real, s=100, c='r', marker='o', zorder=10, label='Lineal')
plt.scatter(pred_ho.ridge, pred_ho.real, s=100, c='g', marker='+', zorder=10, label='Ridge')
plt.scatter(pred_ho.lasso, pred_ho.real, s=100, c='purple', marker='v', zorder=10, label='Lasso')
plt.xlabel("Predicciones ", size=12)
plt.ylabel("Valores reales House precio en USD por m2", size=12)
plt.legend()

plt.show()

Para store solo tiene una propiedad y no hicimos el grafico

In [ ]:
prediccion_sample_caba( df = df_store_sample, scaler = scaler_st, linear_model = modelo_lr_st, ridge_model = modelo_ridge_st, lasso_model = modelo_lasso_st)